In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from astroquery.simbad import Simbad


In [3]:
customSimbad = Simbad()
customSimbad.add_votable_fields('ra')
customSimbad.add_votable_fields('dec')

In [4]:
def plot_cov_ellipse(cov, pos, nstd=1, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the 
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwargs)
    
    print(theta)

    ax.add_artist(ellip)
    return ellip

In [5]:
import kplr

client = kplr.API()

In [6]:
kicid = '8462852'
kic = client.star(kicid)

ra = kic.kic_degree_ra
dec = kic.kic_dec



In [7]:
from astropy.coordinates import SkyCoord

In [8]:
coord = SkyCoord('19 27 57.59', '+36 59 26.59', unit=(u.hourangle, u.degree))

In [9]:
from astroquery.vizier import Vizier
from astropy.coordinates import Angle

vquery = Vizier(columns=['Source', 'RA_ICRS', 'DE_ICRS', '_r',
                             'Gmag', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'pmRApmDEcor', 'Plx', 'e_Plx', 'PlxpmRAcor', 'PlxpmDEcor'])

#coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
result = vquery.query_region(coord, radius=4.0*u.arcsec, catalog=["I/345/gaia2"])

print(result)

TableList with 1 tables:
	'0:I/345/gaia2' with 14 column(s) and 5 row(s) 


In [10]:
print(result[0]['RA_ICRS' , 'DE_ICRS', 'Gmag', 'Plx', 'pmRA', 'pmDE', '_r'][0:10])
closest = np.where(result[0]['_r'] == np.min(result[0]['_r']))

res = result[0][:][closest][0]


#print(res['pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'pmRApmDEcor', 'Gmag'])
#print(res['Plx', 'e_Plx', 'PlxpmRAcor', 'PlxpmDEcor'])

x = np.array([res['Plx'],res['pmRA'],res['pmDE']])
xerr = np.array([res['e_Plx'],res['e_pmRA'],res['e_pmDE']])

print(x)
print(xerr)

    RA_ICRS         DE_ICRS       Gmag     Plx        pmRA      pmDE     _r  
      deg             deg         mag      mas      mas / yr  mas / yr arcsec
--------------- --------------- ------- ---------- --------- --------- ------
291.98996100410  36.98984313405 15.9832     0.6189    -2.907    -4.926  3.177
291.98992413382  36.99132289949 15.4079     0.5693    -3.320    -4.321  2.154
291.98995212590  36.99061674157 14.1022     0.8096    -3.018    -4.351  0.393
291.99105383387  36.99083875761 18.3997     0.1870    -4.679    -3.141  3.159
291.98956166770  36.99124429569 17.2850     0.6689     0.127    -3.901  2.198
[ 0.8096 -3.018  -4.351 ]
[0.0177 0.028  0.036 ]


In [11]:
print(len(result[0]))

5


In [13]:
from astroquery.vizier import Vizier
from astropy.coordinates import Angle
from tqdm import tnrange

vquery = Vizier(columns=['Source', 'RA_ICRS', 'DE_ICRS', '_r',
                             'Gmag', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'pmRApmDEcor', 'Plx', 'e_Plx', 'PlxpmRAcor', 'PlxpmDEcor'])


def find_comps():
    a = np.loadtxt('KIC_Gaia_RaDec.txt', dtype=str)
    res = np.zeros(len(a))
    resmag = np.zeros((len(a), 6))
    for i in tnrange(len(a)):
        line = a[i]
        ra = line[0] + ' ' + line[1] + ' ' + line[2]
        dec = line[3] + ' ' + line[4] + ' ' + line[5]
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.degree))
        result = vquery.query_region(coord, radius=6.0*u.arcsec, catalog=["I/345/gaia2"])
        res[i] = len(result[0])
        resmag[i,0:len(result[0])] = result[0]['Gmag'].tolist()
        #print(resmag[i])
        
    return res, resmag

def find_comps_random():
    np.random.seed(30709)
    res = np.zeros(5329)
    for i in tnrange(5329):
        ra  = np.random.uniform(15) + 285
        dec = np.random.uniform(14) + 37
        #print(ra, dec)
        coord = SkyCoord(ra, dec, unit=(u.degree, u.degree))
        result = vquery.query_region(coord, radius=6.0*u.arcsec, catalog=["I/345/gaia2"])
        #print(len(result[0]))
        try:
            res[i] = len(result[0])
        except:
            res[i] = 0
        
    return res

#coord = SkyCoord('19 27 57.59', '+36 59 26.59', unit=(u.hourangle, u.degree))

In [14]:
res_arr, mag_arr = find_comps()
#res_arr_2 = find_comps_random()

In [15]:
res_arr
mag_arr

array([[15.9832, 15.4079, 14.1022, 18.3997, 17.285 ,  0.    ],
       [14.4522,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [14.6248,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       ...,
       [13.3298,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [13.3145,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [18.9117, 14.4802,  0.    ,  0.    ,  0.    ,  0.    ]])

In [16]:
for i in range(len(mag_arr)):
    mag_arr[i] = np.sort(mag_arr[i])
    while mag_arr[i,0] == 0:
        mag_arr[i] = np.roll(mag_arr[i], -1)
    print(mag_arr[i])

[14.1022 15.4079 15.9832 17.285  18.3997  0.    ]
[14.4522  0.      0.      0.      0.      0.    ]
[14.6248  0.      0.      0.      0.      0.    ]
[14.8559  0.      0.      0.      0.      0.    ]
[14.581  0.     0.     0.     0.     0.   ]
[14.2852  0.      0.      0.      0.      0.    ]
[14.6929  0.      0.      0.      0.      0.    ]
[14.2921  0.      0.      0.      0.      0.    ]
[14.8355  0.      0.      0.      0.      0.    ]
[14.8534  0.      0.      0.      0.      0.    ]
[14.7315 18.4199 19.4799  0.      0.      0.    ]
[14.731  0.     0.     0.     0.     0.   ]
[13.9456 20.8518  0.      0.      0.      0.    ]
[14.1565  0.      0.      0.      0.      0.    ]
[14.8745  0.      0.      0.      0.      0.    ]
[14.7696  0.      0.      0.      0.      0.    ]
[12.3124  0.      0.      0.      0.      0.    ]
[13.3856  0.      0.      0.      0.      0.    ]
[14.6881 19.4336  0.      0.      0.      0.    ]
[14.9433  0.      0.      0.      0.      0.    ]
[14.4191  0.

[14.7534  0.      0.      0.      0.      0.    ]
[12.6721 19.3613  0.      0.      0.      0.    ]
[14.7235  0.      0.      0.      0.      0.    ]
[13.9041  0.      0.      0.      0.      0.    ]
[14.3096  0.      0.      0.      0.      0.    ]
[9.4727 0.     0.     0.     0.     0.    ]
[13.8992  0.      0.      0.      0.      0.    ]
[13.2437  0.      0.      0.      0.      0.    ]
[14.4352 17.3685  0.      0.      0.      0.    ]
[14.1649 19.5769  0.      0.      0.      0.    ]
[14.8927  0.      0.      0.      0.      0.    ]
[14.2806  0.      0.      0.      0.      0.    ]
[14.5843 20.8036  0.      0.      0.      0.    ]
[14.4224  0.      0.      0.      0.      0.    ]
[14.657  18.0866  0.      0.      0.      0.    ]
[14.889  0.     0.     0.     0.     0.   ]
[13.6778  0.      0.      0.      0.      0.    ]
[14.9703  0.      0.      0.      0.      0.    ]
[14.9324  0.      0.      0.      0.      0.    ]
[14.6455  0.      0.      0.      0.      0.    ]
[13.522  0. 

[14.8745 21.0055  0.      0.      0.      0.    ]
[14.7882 18.1816  0.      0.      0.      0.    ]
[14.4438  0.      0.      0.      0.      0.    ]
[14.0001 17.5318 20.8819  0.      0.      0.    ]
[14.3517 19.7141  0.      0.      0.      0.    ]
[14.943  0.     0.     0.     0.     0.   ]
[14.1149  0.      0.      0.      0.      0.    ]
[14.2372  0.      0.      0.      0.      0.    ]
[14.6765 19.2583  0.      0.      0.      0.    ]
[14.9553 18.5732  0.      0.      0.      0.    ]
[14.8749  0.      0.      0.      0.      0.    ]
[14.8509  0.      0.      0.      0.      0.    ]
[14.3038 20.8112  0.      0.      0.      0.    ]
[14.6335 19.835   0.      0.      0.      0.    ]
[14.3022  0.      0.      0.      0.      0.    ]
[14.5461 20.4584  0.      0.      0.      0.    ]
[14.9685  0.      0.      0.      0.      0.    ]
[14.859  19.3262  0.      0.      0.      0.    ]
[14.8469  0.      0.      0.      0.      0.    ]
[14.4294 19.6091  0.      0.      0.      0.    ]
[14.04

[13.8983 18.6033  0.      0.      0.      0.    ]
[12.741  14.7683  0.      0.      0.      0.    ]
[12.5101  0.      0.      0.      0.      0.    ]
[14.9232 19.3036  0.      0.      0.      0.    ]
[14.2772 17.0733  0.      0.      0.      0.    ]
[14.0554  0.      0.      0.      0.      0.    ]
[14.1351 20.7866  0.      0.      0.      0.    ]
[14.8442  0.      0.      0.      0.      0.    ]
[14.3993  0.      0.      0.      0.      0.    ]
[14.3406 20.3665  0.      0.      0.      0.    ]
[14.6689  0.      0.      0.      0.      0.    ]
[14.4574 17.2622 17.4968 20.028   0.      0.    ]
[14.7052 19.0181  0.      0.      0.      0.    ]
[14.2775 19.5299  0.      0.      0.      0.    ]
[14.1724 20.5647  0.      0.      0.      0.    ]
[14.5174 19.6518  0.      0.      0.      0.    ]
[14.7727 17.1607  0.      0.      0.      0.    ]
[14.5072 19.9327  0.      0.      0.      0.    ]
[15.2389 15.7095  0.      0.      0.      0.    ]
[14.9364 18.2397  0.      0.      0.      0.    ]


[14.8293  0.      0.      0.      0.      0.    ]
[14.8198  0.      0.      0.      0.      0.    ]
[14.5802  0.      0.      0.      0.      0.    ]
[14.5514  0.      0.      0.      0.      0.    ]
[14.3513  0.      0.      0.      0.      0.    ]
[11.4938  0.      0.      0.      0.      0.    ]
[14.3935  0.      0.      0.      0.      0.    ]
[14.7638 16.8154  0.      0.      0.      0.    ]
[14.7284  0.      0.      0.      0.      0.    ]
[13.2848  0.      0.      0.      0.      0.    ]
[14.5247 19.8022  0.      0.      0.      0.    ]
[14.2066 18.3346 18.4743  0.      0.      0.    ]
[14.3111  0.      0.      0.      0.      0.    ]
[12.9575 19.9443  0.      0.      0.      0.    ]
[12.6268 19.4784  0.      0.      0.      0.    ]
[14.6074 16.6987  0.      0.      0.      0.    ]
[14.8184  0.      0.      0.      0.      0.    ]
[14.6043  0.      0.      0.      0.      0.    ]
[14.5686  0.      0.      0.      0.      0.    ]
[14.1544 18.7942  0.      0.      0.      0.    ]


...

[14.6991  0.      0.      0.      0.      0.    ]
[14.8888  0.      0.      0.      0.      0.    ]
[12.7106  0.      0.      0.      0.      0.    ]
[14.8404  0.      0.      0.      0.      0.    ]
[13.3529  0.      0.      0.      0.      0.    ]
[14.1272  0.      0.      0.      0.      0.    ]
[14.2925 16.6507 17.9176  0.      0.      0.    ]
[10.5783  0.      0.      0.      0.      0.    ]
[13.6459  0.      0.      0.      0.      0.    ]
[13.2626  0.      0.      0.      0.      0.    ]
[13.6398  0.      0.      0.      0.      0.    ]
[14.6291  0.      0.      0.      0.      0.    ]
[14.8037  0.      0.      0.      0.      0.    ]
[14.7184  0.      0.      0.      0.      0.    ]
[14.262  19.8242  0.      0.      0.      0.    ]
[14.9791  0.      0.      0.      0.      0.    ]
[14.1633  0.      0.      0.      0.      0.    ]
[14.8078  0.      0.      0.      0.      0.    ]
[14.7282  0.      0.      0.      0.      0.    ]
[14.9219  0.      0.      0.      0.      0.    ]


[14.6732 20.1191  0.      0.      0.      0.    ]
[14.3571  0.      0.      0.      0.      0.    ]
[14.6644  0.      0.      0.      0.      0.    ]
[14.2472  0.      0.      0.      0.      0.    ]
[14.9146  0.      0.      0.      0.      0.    ]
[14.6289  0.      0.      0.      0.      0.    ]
[13.4364  0.      0.      0.      0.      0.    ]
[14.3844  0.      0.      0.      0.      0.    ]
[14.7567  0.      0.      0.      0.      0.    ]
[14.5682  0.      0.      0.      0.      0.    ]
[14.8578 19.8779 20.9729  0.      0.      0.    ]
[12.9095  0.      0.      0.      0.      0.    ]
[14.825  0.     0.     0.     0.     0.   ]
[14.0234  0.      0.      0.      0.      0.    ]
[14.7144  0.      0.      0.      0.      0.    ]
[12.5934  0.      0.      0.      0.      0.    ]
[14.9894 17.9398 20.9312  0.      0.      0.    ]
[14.0262  0.      0.      0.      0.      0.    ]
[14.9897  0.      0.      0.      0.      0.    ]
[14.665  0.     0.     0.     0.     0.   ]
[14.9459  0.

[13.4471  0.      0.      0.      0.      0.    ]
[14.0272  0.      0.      0.      0.      0.    ]
[12.7614  0.      0.      0.      0.      0.    ]
[14.7342  0.      0.      0.      0.      0.    ]
[14.7597  0.      0.      0.      0.      0.    ]
[13.3732  0.      0.      0.      0.      0.    ]
[14.8455  0.      0.      0.      0.      0.    ]
[13.773  0.     0.     0.     0.     0.   ]
[14.7215  0.      0.      0.      0.      0.    ]
[14.8535  0.      0.      0.      0.      0.    ]
[14.8229 20.2129  0.      0.      0.      0.    ]
[14.8804  0.      0.      0.      0.      0.    ]
[11.891  0.     0.     0.     0.     0.   ]
[13.8485  0.      0.      0.      0.      0.    ]
[14.5269  0.      0.      0.      0.      0.    ]
[14.9372 18.9129  0.      0.      0.      0.    ]
[14.8701 20.7307  0.      0.      0.      0.    ]
[14.793  0.     0.     0.     0.     0.   ]
[12.9533  0.      0.      0.      0.      0.    ]
[13.8914  0.      0.      0.      0.      0.    ]
[13.2924  0.      

[14.8878  0.      0.      0.      0.      0.    ]
[14.3952  0.      0.      0.      0.      0.    ]
[14.8554  0.      0.      0.      0.      0.    ]
[14.7134  0.      0.      0.      0.      0.    ]
[14.8239  0.      0.      0.      0.      0.    ]
[14.4242 15.2425  0.      0.      0.      0.    ]
[13.6748 19.409   0.      0.      0.      0.    ]
[14.9455  0.      0.      0.      0.      0.    ]
[11.9071 13.7703  0.      0.      0.      0.    ]
[14.5573  0.      0.      0.      0.      0.    ]
[14.6281  0.      0.      0.      0.      0.    ]
[14.2824 15.9427  0.      0.      0.      0.    ]
[14.4995 19.8128  0.      0.      0.      0.    ]
[14.3514 14.9823  0.      0.      0.      0.    ]
[14.6615  0.      0.      0.      0.      0.    ]
[14.5682  0.      0.      0.      0.      0.    ]
[14.2989  0.      0.      0.      0.      0.    ]
[13.4809  0.      0.      0.      0.      0.    ]
[14.8389  0.      0.      0.      0.      0.    ]
[14.5621  0.      0.      0.      0.      0.    ]


[13.6361  0.      0.      0.      0.      0.    ]
[14.7809 16.8653  0.      0.      0.      0.    ]
[14.3205  0.      0.      0.      0.      0.    ]
[14.1323  0.      0.      0.      0.      0.    ]
[14.4449 21.112   0.      0.      0.      0.    ]
[14.0602  0.      0.      0.      0.      0.    ]
[14.6572 20.8173  0.      0.      0.      0.    ]
[14.0776 20.1398  0.      0.      0.      0.    ]
[14.7153 19.2427  0.      0.      0.      0.    ]
[14.1304  0.      0.      0.      0.      0.    ]
[14.8995  0.      0.      0.      0.      0.    ]
[14.9248  0.      0.      0.      0.      0.    ]
[14.4002  0.      0.      0.      0.      0.    ]
[13.2604  0.      0.      0.      0.      0.    ]
[14.6105  0.      0.      0.      0.      0.    ]
[13.7619  0.      0.      0.      0.      0.    ]
[14.8659  0.      0.      0.      0.      0.    ]
[14.1199  0.      0.      0.      0.      0.    ]
[14.6326 17.1978  0.      0.      0.      0.    ]
[14.0777  0.      0.      0.      0.      0.    ]


In [17]:
plt.hist(res_arr_2, [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])

NameError: name 'res_arr_2' is not defined

In [ ]:
np.sqrt(4.0**2*(1-len(res_arr_2[res_arr_2 == 0])/len(res_arr[res_arr == 1])))

In [18]:
len(res_arr[res_arr==3])

326

In [19]:
a = np.loadtxt('KIC_Gaia_RaDec.txt', dtype=str)

In [20]:
kic = np.array(a[:,-1], dtype=int)

In [21]:
np.savetxt('n_comps_updated_6arcsec.txt', np.array([kic, res_arr, mag_arr[:,0], mag_arr[:,1],
                                            mag_arr[:,2], mag_arr[:,3], mag_arr[:,4], mag_arr[:,5]]).T, 
          fmt='%10i %1i %.3f %.3f %.3f %.3f %.3f %.3f')

In [135]:
import os
os.getcwd()

'/home/bmontet/research/tools/Gaiaxmatch'